# predict

> Common operations around the core datastructures for running a sim

In [ ]:
#| default_exp predict

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

from flax.struct import dataclass as flax_dataclass
import jax.numpy as jnp
from typing import Optional
from chewc.population import Population

In [ ]:
#| export
"""
GBLUP (Genomic Best Linear Unbiased Prediction) implementation for chewc library.

This module provides functions for genomic prediction using the GBLUP methodology,
which is a standard approach in genomic selection and animal breeding.
"""

from typing import Optional, Dict
import jax
import jax.numpy as jnp
from jax.numpy.linalg import solve, inv, pinv
from flax.struct import dataclass as flax_dataclass

from chewc.population import Population

@flax_dataclass(frozen=True)
class PredictionResults:
    """
    A container for the results of a genomic prediction.
    """
    ids: jnp.ndarray
    ebv: jnp.ndarray
    pev: Optional[jnp.ndarray] = None
    reliability: Optional[jnp.ndarray] = None
    fixed_effects: Optional[jnp.ndarray] = None
    h2_used: Optional[float] = None
    var_components: Optional[Dict] = None



In [ ]:
#| export
from functools import partial
from typing import Optional, Dict, Tuple
from jax.scipy.sparse.linalg import cg
from jax import lax

# ==================================================
# --- Core Matrix Calculation Functions ---
# ==================================================
@partial(jax.jit, static_argnames=('n_ind',))
def _jit_calc_a_inverse(mother_iids: jnp.ndarray, father_iids: jnp.ndarray, n_ind: int) -> jnp.ndarray:
    initial_A_inv = jnp.zeros((n_ind, n_ind))
    def loop_body(i, A_inv):
        sire_iid, dam_iid = father_iids[i], mother_iids[i]
        case_index = (sire_iid != -1) + 2 * (dam_iid != -1)
        def case_0(mat): return mat.at[i, i].add(1.0)
        def case_1(mat): return mat.at[i, i].add(4/3.).at[sire_iid, sire_iid].add(1/3.).at[i, sire_iid].add(-2/3.).at[sire_iid, i].add(-2/3.)
        def case_2(mat): return mat.at[i, i].add(4/3.).at[dam_iid, dam_iid].add(1/3.).at[i, dam_iid].add(-2/3.).at[dam_iid, i].add(-2/3.)
        def case_3(mat): return (mat.at[i, i].add(2.0).at[sire_iid, sire_iid].add(0.5).at[dam_iid, dam_iid].add(0.5)
                                 .at[sire_iid, dam_iid].add(0.5).at[dam_iid, sire_iid].add(0.5).at[i, sire_iid].add(-1.0)
                                 .at[sire_iid, i].add(-1.0).at[i, dam_iid].add(-1.0).at[dam_iid, i].add(-1.0))
        return lax.switch(case_index, [case_0, case_1, case_2, case_3], A_inv)
    return lax.fori_loop(0, n_ind, loop_body, initial_A_inv)

def calc_a_inverse_matrix_pedigree_jax(pop: Population) -> jnp.ndarray:
    n_ind = pop.nInd
    id_to_iid_map = jnp.full(pop.id.max() + 2, -1, dtype=jnp.int32).at[pop.id].set(pop.iid)
    mother_iids = jnp.where(pop.mother < 0, -1, id_to_iid_map[pop.mother.clip(min=0)])
    father_iids = jnp.where(pop.father < 0, -1, id_to_iid_map[pop.father.clip(min=0)])
    return _jit_calc_a_inverse(mother_iids, father_iids, n_ind)

@jax.jit
def calc_g_matrix(geno_dosage: jnp.ndarray) -> jnp.ndarray:
    """
    Calculates the genomic relationship matrix (G).
    --- FIX ---
    The original epsilon was too small to prevent the G matrix from becoming
    ill-conditioned in later generations. Increasing it slightly adds a larger
    "ridge" to the diagonal, making the matrix more stable and invertible.
    We also blend G with the identity matrix, which is a common technique to
    improve the conditioning of G.
    """
    n_ind, n_markers = geno_dosage.shape
    p = jnp.mean(geno_dosage, axis=0) / 2.0
    P = 2 * p
    M = geno_dosage - P
    denominator = 2 * jnp.sum(p * (1 - p))
    denominator = jnp.where(denominator == 0, 1e-8, denominator)
    G_raw = (M @ M.T) / denominator

    # --- FIX ---
    # Blend G with the identity matrix to improve conditioning.
    # 0.99 is a common blending factor.
    G = 0.99 * G_raw + 0.01 * jnp.identity(n_ind)
    # Add a slightly larger epsilon for numerical stability.
    epsilon = 1e-3
    return G + jnp.identity(n_ind) * epsilon

@partial(jax.jit, static_argnames='n_ind')
def _mme_solver_cg(pheno: jnp.ndarray, train_mask: jnp.ndarray, K_inv: jnp.ndarray, h2: float, n_ind: int) -> Tuple[jnp.ndarray, jnp.ndarray]:
    """
    Mixed model equation (MME) solver using the conjugate gradient algorithm.
    --- FIX ---
    The preconditioner is improved by adding a small offset to the diagonal elements.
    This prevents division by very small numbers if any diagonal elements of K_inv
    are close to zero, which can happen with ill-conditioned matrices.
    """
    alpha = (1.0 - h2) / h2
    y = jnp.nan_to_num(pheno.flatten())
    train_mask_float = train_mask.astype(jnp.float32)
    n_train = jnp.sum(train_mask_float)

    def lhs_matvec(solution_vector):
        beta, u = solution_vector[0], solution_vector[1:]
        row1 = n_train * beta + jnp.sum(u * train_mask_float)
        row2 = train_mask_float * beta + u * train_mask_float + (K_inv @ u) * alpha
        return jnp.concatenate([jnp.array([row1]), row2])

    rhs = jnp.concatenate([
        jnp.array([jnp.sum(y * train_mask_float)]),
        y * train_mask_float
    ])

    # --- FIX ---
    # A more robust diagonal preconditioner. Adding a small constant (1e-6)
    # ensures that we don't divide by zero, even if some diagonal
    # elements are very small.
    M_diag = jnp.concatenate([
        jnp.array([n_train]),
        train_mask_float + jnp.diag(K_inv) * alpha + 1e-6
    ])
    M_diag = jnp.maximum(M_diag, 1e-6)
    # Increase the tolerance and max iterations for the CG solver for more stability
    solutions, _ = cg(lhs_matvec, rhs, M=lambda x: x / M_diag, tol=1e-5, maxiter=2*n_ind)
    return solutions[0:1], solutions[1:]

# ==================================================
# --- Prediction Functions (ABLUP and GBLUP) ---
# ==================================================
def mme_predict_ablup(pop: Population, h2: float, trait_idx: int = 0) -> PredictionResults:
    pheno = pop.pheno[:, trait_idx:trait_idx+1]
    n_ind = pop.nInd
    train_mask = ~jnp.isnan(pheno.flatten())
    if jnp.sum(train_mask) == 0: raise ValueError("No individuals with phenotypes.")
    K_inv = calc_a_inverse_matrix_pedigree_jax(pop)
    # --- FIX ---
    # Call the unified MME solver with the inverse of the pedigree relationship matrix (A_inv).
    fixed_effects, all_ebv = _mme_solver_cg(pheno, train_mask, K_inv, h2, n_ind)
    return PredictionResults(ids=pop.id, ebv=all_ebv.reshape(-1, 1), fixed_effects=fixed_effects, h2_used=h2)

def mme_predict_gblup(pop: Population, h2: float, trait_idx: int = 0) -> PredictionResults:
    """
    Predicts Estimated Breeding Values (EBVs) using Genomic Best Linear Unbiased Prediction (GBLUP).
    This version includes jax.debug.print statements to diagnose NaN issues.
    """
    pheno = pop.pheno[:, trait_idx:trait_idx+1]
    n_ind = pop.nInd
    train_mask = ~jnp.isnan(pheno.flatten())

    if jnp.sum(train_mask) == 0:
        raise ValueError("No individuals with phenotypes.")

    # --- JAX DEBUG STATEMENTS ---
    jax.debug.print("--- GBLUP Debug Start ---")
    jax.debug.print("h2: {h2}", h2=h2)
    jax.debug.print("Number of individuals: {n}", n=n_ind)
    jax.debug.print("Number of phenotyped individuals: {p}", p=jnp.sum(train_mask))
    jax.debug.print("Pheno shape: {s}, contains NaNs: {n}", s=pheno.shape, n=jnp.any(jnp.isnan(pheno)))
    # --- END DEBUG ---

    G = calc_g_matrix(pop.dosage)

    # --- JAX DEBUG STATEMENTS ---
    jax.debug.print("G matrix shape: {s}", s=G.shape)
    jax.debug.print("G matrix contains NaNs: {n}", n=jnp.any(jnp.isnan(G)))
    jax.debug.print("G matrix diagonal min/max: {min}/{max}", min=jnp.min(jnp.diag(G)), max=jnp.max(jnp.diag(G)))
    # Check the condition number to assess if G is ill-conditioned (close to singular)
    cond_g = jnp.linalg.cond(G)
    jax.debug.print("G matrix condition number: {c}", c=cond_g)
    # --- END DEBUG ---

    # Invert the G matrix. A large condition number might cause NaNs here.
    G_inv = jnp.linalg.inv(G)

    # --- JAX DEBUG STATEMENTS ---
    jax.debug.print("G_inv matrix contains NaNs: {n}", n=jnp.any(jnp.isnan(G_inv)))
    # --- END DEBUG ---

    fixed_effects, all_ebv = _mme_solver_cg(pheno, train_mask, G_inv, h2, n_ind)

    # --- JAX DEBUG STATEMENTS ---
    jax.debug.print("Solved fixed_effects contains NaNs: {n}", n=jnp.any(jnp.isnan(fixed_effects)))
    jax.debug.print("Solved all_ebv contains NaNs: {n}", n=jnp.any(jnp.isnan(all_ebv)))
    jax.debug.print("--- GBLUP Debug End ---")
    # --- END DEBUG ---

    return PredictionResults(ids=pop.id, ebv=all_ebv.reshape(-1, 1), fixed_effects=fixed_effects, h2_used=h2)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()